In [2]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import chain, combinations
from collections import Counter


from collections import defaultdict
def powerset(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    result = []
    for i in range(1 << x):
        result.append([ss for mask, ss in zip(masks, s) if i & mask])
    return result

In [3]:
def unitary_cayley(f, R):
    """
    This function return the unitary Cayley graph on F_q[x]/f
    """
    
    n = f.degree()-1
    vertice = elements_quotient_ring = list(R.polynomials(max_degree=n))
    n = len(vertice)
    g=nx.empty_graph(n)
    for i in range(0,n):
        for j in range(0,n):
            if gcd(vertice[i]-vertice[j],f) ==1:
                g.add_edge(i,j)
    return g  

def gcd_cayley(f, R, D):
    """
    return the gcd graph G_f(D)
    """
    n = f.degree()-1
    vertice = elements_quotient_ring = list(R.polynomials(max_degree=n))
    n = len(vertice)
    g=nx.empty_graph(n)
    for i in range(0,n):
        for j in range(0,n):
            if (gcd(vertice[i]-vertice[j],f) in D) or (gcd(vertice[j]-vertice[i],f) in D):
                g.add_edge(i,j)
    return g 

def abstract_gcd_cayley(f, R, D):
    n = f.degree()-1
    vertice = elements_quotient_ring = list(R.polynomials(max_degree=n))
    n = len(vertice)
    g=nx.empty_graph(vertice)
    for i in vertice:
        for j in vertice:
            if gcd(i-j,f) in D:
                g.add_edge(i,j)
    return g 

def induced_graph(g,v):
    n = len(v)
    induced_g=nx.empty_graph(n)
    for i in range(n):
        for j in range(n):
            if v[j] in g.neighbors(v[i]):
                induced_g.add_edge(i,j)
    return induced_g 

def new_gcd(f,g):
    if f % g ==0:
        return 0
    else:
        return gcd(f,g)
    
def raw_char_poly(g):
    adj_matrix = nx.adjacency_matrix(g).todense()
    matrix_size = g.number_of_nodes()
    matrix = adj_matrix
    f = MatrixSpace(IntegerRing(),matrix_size)(matrix).charpoly()
    return f
    
    
def char_poly_gcd_graph(g):
    adj_matrix = nx.adjacency_matrix(g).todense()
    matrix_size = g.number_of_nodes()
    matrix = adj_matrix
    f = MatrixSpace(IntegerRing(),matrix_size)(matrix).charpoly()
    return f.factor()

def all_divisors(f):
    n = f.degree()-1
    vertice = elements_quotient_ring = list(R.polynomials(max_degree=n))
    result = []
    for item in vertice:
        if item !=0 and f % item == 0 and item.is_monic():
            result.append(item)
    return result

def all_divisors_new(f):
    n = f.degree()
    vertice = elements_quotient_ring = list(R.polynomials(max_degree=n))
    result = []
    for item in vertice:
        if item !=0 and f % item == 0 and item.is_monic():
            result.append(item)
    return result  

def radical_polynomial(g):
    F = raw_char_poly(g)
    factors = F.factor()
    result = 1
    for factor in factors:
        result *= factor[0]
    return result    
    

In [4]:
def euler_function(f,R):
    n = f.degree()-1
    vertice = elements_quotient_ring = list(R.polynomials(max_degree=n))
    if n != -1: 
        count = 0 
        for item in vertice:
            if gcd(f,item)==1:
                count +=1 
        return count 
    else:
        return 1
        
        

def moebius_function(f):
    factors = f.factor()
    value = 0
    for factor in factors:
        if factor[1] !=1:
            return 0
        else:
            value +=1
    return (-1)**value        
        

def ramanujan_sum(g,f,R):
    """
    calculate the Ramanujan sum c(g,f) 
    """
    if g % f ==0:
        return euler_function(f,R)
    h = gcd(f,g)
    t, _ = f.quo_rem(h)
    value  = (moebius_function(t) * euler_function(f,R))//(euler_function(t,R))
    return value

def spectra_gcd_graph(f, R, D):
    n = f.degree()-1
    vertice = elements_quotient_ring = list(R.polynomials(max_degree=n))
    new_D = [f.quo_rem(g)[0] for g in D]
    spectra = []
    for v in vertice:
        count = 0 
        for item in new_D:
            count += ramanujan_sum(v, item, R)
        spectra.append(count)
    return Counter(spectra)
    
def energy_gcd_graph(f, R, D):
    n = f.degree()-1
    vertice = elements_quotient_ring = list(R.polynomials(max_degree=n))
    new_D = [f.quo_rem(g)[0] for g in D]
    spectra = []
    for v in vertice:
        count = 0 
        for item in new_D:
            count += ramanujan_sum(v, item, R)
        spectra.append(count)
    abs_spectra = [abs(item) for item in spectra]    
    return abs_spectra

def gcd_determinant(f,R):
    f_divisors = all_divisors_new(f)
    M = len(f_divisors)
    A = matrix.identity(M)
    for i in range(M):
        for j in range(M):
            A[i,j] = ramanujan_sum(f_divisors[i], f_divisors[j],R)
    return A

def existence_odd_cycle(g):
    result =[]
    for cycle in cycles:
        if len(cycle) % 2 == 1 and len(cycle)>3:
            result.append(cycle)
            print(cycle)
            break
    return len(result) > 0 
    

Test the function that calculates that characteristic polynomial of a gcd-graph

In [4]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
f = x*(x+1)^2
D = [1,x+1, (x+1)^2]
g = abstract_gcd_cayley(f,R,D)
h= char_poly_gcd_graph(g)
h

(x - 18) * (x + 9)^2 * x^24

Test the function that calculate Ramanujan sums

In [5]:
spectra_gcd_graph(f,R,D)

Counter({0: 24, -9: 2, 18: 1})

Conclusion: Both methods give the same answer for the spectrum of the gcd graph $G_{f}(D)$ with $f=x*(x+1)^2 \in \mathbb{F}_3[x]$
and $D=[1, x+1, (x+1)^2]$. 

# We test these functions for other gcd-graphs

In [6]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
f = x*(x^2+1)
D = [1,x]
g = abstract_gcd_cayley(f,R,D)
h= char_poly_gcd_graph(g)
print(h)
print(spectra_gcd_graph(f,R,D))

(x - 24) * (x + 3)^8 * x^18
Counter({0: 18, -3: 8, 24: 1})


In [7]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
f = x^5
D = [1,x^4]
g = abstract_gcd_cayley(f,R,D)
h= char_poly_gcd_graph(g)
print(h)
print(spectra_gcd_graph(f,R,D))

(x - 164) * (x + 79)^2 * (x - 2)^78 * (x + 1)^162
Counter({-1: 162, 2: 78, -79: 2, 164: 1})


In [12]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 5
f = x^n
D = [1, x, x^4]
g = abstract_gcd_cayley(f,R,D)
h= char_poly_gcd_graph(g)
print(h)
print(spectra_gcd_graph(f,R,D))

(x - 218) * (x + 25)^8 * (x - 2)^72 * (x + 1)^162
Counter({-1: 162, 2: 72, -25: 8, 218: 1})


In [15]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 5
f = x^n
D = [1, x^3, x^4]
g = abstract_gcd_cayley(f,R,D)
h= char_poly_gcd_graph(g)
print(h)
print(spectra_gcd_graph(f,R,D))

(x - 170) * (x + 73)^2 * (x - 8)^24 * (x + 1)^216
Counter({-1: 216, 8: 24, -73: 2, 170: 1})


# Spectrum of $G_{f}(D)$ where $f=x^5 \in \mathbb{F}_3[x]$ and $D$ is a subset of $Div(f)$.

In [5]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 5
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
result

[[[1], Counter({0: 240, -81: 2, 162: 1})],
 [[x], Counter({0: 234, -27: 6, 54: 3})],
 [[1, x], Counter({0: 234, -27: 8, 216: 1})],
 [[x^2], Counter({0: 216, -9: 18, 18: 9})],
 [[1, x^2], Counter({0: 216, -9: 18, 18: 6, -63: 2, 180: 1})],
 [[x, x^2], Counter({0: 216, -9: 24, 72: 3})],
 [[1, x, x^2], Counter({0: 216, -9: 26, 234: 1})],
 [[x^3], Counter({0: 162, -3: 54, 6: 27})],
 [[1, x^3], Counter({0: 162, -3: 54, 6: 24, -75: 2, 168: 1})],
 [[x, x^3], Counter({0: 162, -3: 54, 6: 18, -21: 6, 60: 3})],
 [[1, x, x^3], Counter({0: 162, -3: 54, 6: 18, -21: 8, 222: 1})],
 [[x^2, x^3], Counter({0: 162, -3: 72, 24: 9})],
 [[1, x^2, x^3], Counter({0: 162, -3: 72, 24: 6, -57: 2, 186: 1})],
 [[x, x^2, x^3], Counter({0: 162, -3: 78, 78: 3})],
 [[1, x, x^2, x^3], Counter({0: 162, -3: 80, 240: 1})],
 [[x^4], Counter({-1: 162, 2: 81})],
 [[1, x^4], Counter({-1: 162, 2: 78, -79: 2, 164: 1})],
 [[x, x^4], Counter({-1: 162, 2: 72, -25: 6, 56: 3})],
 [[1, x, x^4], Counter({-1: 162, 2: 72, -25: 8, 218: 1})

# Find $D$ such that $-1$ is an eigenvalue of $G_f(D)$

In [19]:
count = 0 
for item in result:
    if -1 in item[1]:
        print(item)
        count +=1
print(count)

[[x^4], Counter({-1: 162, 2: 81})]
[[1, x^4], Counter({-1: 162, 2: 78, -79: 2, 164: 1})]
[[x, x^4], Counter({-1: 162, 2: 72, -25: 6, 56: 3})]
[[1, x, x^4], Counter({-1: 162, 2: 72, -25: 8, 218: 1})]
[[x^2, x^4], Counter({-1: 162, 2: 54, -7: 18, 20: 9})]
[[1, x^2, x^4], Counter({-1: 162, 2: 54, -7: 18, 20: 6, -61: 2, 182: 1})]
[[x, x^2, x^4], Counter({-1: 162, 2: 54, -7: 24, 74: 3})]
[[1, x, x^2, x^4], Counter({-1: 162, 2: 54, -7: 26, 236: 1})]
[[x^3, x^4], Counter({-1: 216, 8: 27})]
[[1, x^3, x^4], Counter({-1: 216, 8: 24, -73: 2, 170: 1})]
[[x, x^3, x^4], Counter({-1: 216, 8: 18, -19: 6, 62: 3})]
[[1, x, x^3, x^4], Counter({-1: 216, 8: 18, -19: 8, 224: 1})]
[[x^2, x^3, x^4], Counter({-1: 234, 26: 9})]
[[1, x^2, x^3, x^4], Counter({-1: 234, 26: 6, -55: 2, 188: 1})]
[[x, x^2, x^3, x^4], Counter({-1: 240, 80: 3})]
[[1, x, x^2, x^3, x^4], Counter({-1: 242, 242: 1})]
16


# $f= x^3 \in \mathbb{F}_4[x]$

In [21]:
F.<a> = GF(4)
R.<x> = PolynomialRing(F)
n = 4
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
count = 0 
for item in result:
    if -1 in item[1]:
        print(item)
        count +=1
print(count)

[[x^3], Counter({-1: 192, 3: 64})]
[[1, x^3], Counter({-1: 192, 3: 60, -61: 3, 195: 1})]
[[x, x^3], Counter({-1: 192, 3: 48, -13: 12, 51: 4})]
[[1, x, x^3], Counter({-1: 192, 3: 48, -13: 15, 243: 1})]
[[x^2, x^3], Counter({-1: 240, 15: 16})]
[[1, x^2, x^3], Counter({-1: 240, 15: 12, -49: 3, 207: 1})]
[[x, x^2, x^3], Counter({-1: 252, 63: 4})]
[[1, x, x^2, x^3], Counter({-1: 255, 255: 1})]
8


# $f = (x^2+1)^4 \in \mathbb{F}_3[x]$

In [22]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 4
f = (x^2+1)^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
count = 0 
for item in result:
    if -1 in item[1]:
        print(item)
        count +=1
print(count)

[[x^6 + 1], Counter({-1: 5832, 8: 729})]
[[1, x^6 + 1], Counter({-1: 5832, 8: 720, -721: 8, 5840: 1})]
[[x^2 + 1, x^6 + 1], Counter({-1: 5832, 8: 648, -73: 72, 656: 9})]
[[1, x^2 + 1, x^6 + 1], Counter({-1: 5832, 8: 648, -73: 80, 6488: 1})]
[[x^4 + 2*x^2 + 1, x^6 + 1], Counter({-1: 6480, 80: 81})]
[[1, x^4 + 2*x^2 + 1, x^6 + 1], Counter({-1: 6480, 80: 72, -649: 8, 5912: 1})]
[[x^2 + 1, x^4 + 2*x^2 + 1, x^6 + 1], Counter({-1: 6552, 728: 9})]
[[1, x^2 + 1, x^4 + 2*x^2 + 1, x^6 + 1], Counter({-1: 6560, 6560: 1})]
8


# $f=(x^2+x+1)^4 \in \mathbb{F}_2[x]$

In [24]:
F.<a> = GF(2)
R.<x> = PolynomialRing(F)
n = 4
f = (x**2+x+1)^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
count = 0 
for item in result:
    if -1 in item[1]:
        print(item)
        count +=1
print(count)

[[x^6 + x^5 + x^3 + x + 1], Counter({-1: 192, 3: 64})]
[[1, x^6 + x^5 + x^3 + x + 1], Counter({-1: 192, 3: 60, -61: 3, 195: 1})]
[[x^2 + x + 1, x^6 + x^5 + x^3 + x + 1], Counter({-1: 192, 3: 48, -13: 12, 51: 4})]
[[1, x^2 + x + 1, x^6 + x^5 + x^3 + x + 1], Counter({-1: 192, 3: 48, -13: 15, 243: 1})]
[[x^4 + x^2 + 1, x^6 + x^5 + x^3 + x + 1], Counter({-1: 240, 15: 16})]
[[1, x^4 + x^2 + 1, x^6 + x^5 + x^3 + x + 1], Counter({-1: 240, 15: 12, -49: 3, 207: 1})]
[[x^2 + x + 1, x^4 + x^2 + 1, x^6 + x^5 + x^3 + x + 1], Counter({-1: 252, 63: 4})]
[[1, x^2 + x + 1, x^4 + x^2 + 1, x^6 + x^5 + x^3 + x + 1], Counter({-1: 255, 255: 1})]
8


## The list of $D$ such that  of $G_{f}(D)$ has exactly 2 eigenvalues.

In [9]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 4
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
for item in result:
    if len(item[1])==2:
        print(item)

[[x^3], Counter({-1: 54, 2: 27})]
[[x^2, x^3], Counter({-1: 72, 8: 9})]
[[x, x^2, x^3], Counter({-1: 78, 26: 3})]
[[1, x, x^2, x^3], Counter({-1: 80, 80: 1})]


In [10]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 5
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
for item in result:
    if len(item[1])==2:
        print(item)

[[x^4], Counter({-1: 162, 2: 81})]
[[x^3, x^4], Counter({-1: 216, 8: 27})]
[[x^2, x^3, x^4], Counter({-1: 234, 26: 9})]
[[x, x^2, x^3, x^4], Counter({-1: 240, 80: 3})]
[[1, x, x^2, x^3, x^4], Counter({-1: 242, 242: 1})]


In [12]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 4
f = (x^2+1)^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
for item in result:
    if len(item[1])==2:
        print(item)

[[x^6 + 1], Counter({-1: 5832, 8: 729})]
[[x^4 + 2*x^2 + 1, x^6 + 1], Counter({-1: 6480, 80: 81})]
[[x^2 + 1, x^4 + 2*x^2 + 1, x^6 + 1], Counter({-1: 6552, 728: 9})]
[[1, x^2 + 1, x^4 + 2*x^2 + 1, x^6 + 1], Counter({-1: 6560, 6560: 1})]


# Eigenvalue 0

In [8]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 4
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
count = 0 
for item in result:
    if 0 in item[1]:
        print(item)
        count +=1
print(count)

[[], Counter({0: 81})]
[[1], Counter({0: 78, -27: 2, 54: 1})]
[[x], Counter({0: 72, -9: 6, 18: 3})]
[[1, x], Counter({0: 72, -9: 8, 72: 1})]
[[x^2], Counter({0: 54, -3: 18, 6: 9})]
[[1, x^2], Counter({0: 54, -3: 18, 6: 6, -21: 2, 60: 1})]
[[x, x^2], Counter({0: 54, -3: 24, 24: 3})]
[[1, x, x^2], Counter({0: 54, -3: 26, 78: 1})]
8


In [10]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 5
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
count = 0 
for item in result:
    if 0 in item[1]:
        print(item)
        count +=1
print(count)

[[], Counter({0: 243})]
[[1], Counter({0: 240, -81: 2, 162: 1})]
[[x], Counter({0: 234, -27: 6, 54: 3})]
[[1, x], Counter({0: 234, -27: 8, 216: 1})]
[[x^2], Counter({0: 216, -9: 18, 18: 9})]
[[1, x^2], Counter({0: 216, -9: 18, 18: 6, -63: 2, 180: 1})]
[[x, x^2], Counter({0: 216, -9: 24, 72: 3})]
[[1, x, x^2], Counter({0: 216, -9: 26, 234: 1})]
[[x^3], Counter({0: 162, -3: 54, 6: 27})]
[[1, x^3], Counter({0: 162, -3: 54, 6: 24, -75: 2, 168: 1})]
[[x, x^3], Counter({0: 162, -3: 54, 6: 18, -21: 6, 60: 3})]
[[1, x, x^3], Counter({0: 162, -3: 54, 6: 18, -21: 8, 222: 1})]
[[x^2, x^3], Counter({0: 162, -3: 72, 24: 9})]
[[1, x^2, x^3], Counter({0: 162, -3: 72, 24: 6, -57: 2, 186: 1})]
[[x, x^2, x^3], Counter({0: 162, -3: 78, 78: 3})]
[[1, x, x^2, x^3], Counter({0: 162, -3: 80, 240: 1})]
16


# Eigenvalue 1 

In [16]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 5
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
for item in result:
    if 1 in item[1]:
        print(item)

In [23]:
F.<a> = GF(2)
R.<x> = PolynomialRing(F)
n = 2
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
for item in result:
    if 1 in item[1]:
        print(item)

[[x], Counter({1: 2, -1: 2})]


In [28]:
F.<a> = GF(2)
R.<x> = PolynomialRing(F)
n = 3
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
for item in result:
    if 1 in item[1] and 1 in item[0]:
        print(item)

[[1, x^2], Counter({-1: 4, 1: 2, 5: 1, -3: 1})]


In [29]:
F.<a> = GF(2)
R.<x> = PolynomialRing(F)
n = 4
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
for item in result:
    if 1 in item[1] and 1 in item[0]:
        print(item)

[[1, x^3], Counter({-1: 8, 1: 6, 9: 1, -7: 1})]
[[1, x, x^3], Counter({-1: 8, 1: 4, -3: 3, 13: 1})]


In [32]:
F.<a> = GF(2)
R.<x> = PolynomialRing(F)
n = 5
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
count = 0     
for item in result:
    if 1 in item[1] and 1 in item[0]:
        print(item)
        count +=1
print(count)

[[1, x^4], Counter({-1: 16, 1: 14, 17: 1, -15: 1})]
[[1, x, x^4], Counter({-1: 16, 1: 12, -7: 3, 25: 1})]
[[1, x^2, x^4], Counter({-1: 16, 1: 8, -3: 4, 5: 2, 21: 1, -11: 1})]
[[1, x, x^2, x^4], Counter({-1: 16, 1: 8, -3: 7, 29: 1})]
4


In [31]:
F.<a> = GF(2)
R.<x> = PolynomialRing(F)
n = 6
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
count = 0     
for item in result:
    if 1 in item[1] and 1 in item[0]:
        print(item)
        count +=1
print(count)

[[1, x^5], Counter({-1: 32, 1: 30, 33: 1, -31: 1})]
[[1, x, x^5], Counter({-1: 32, 1: 28, -15: 3, 49: 1})]
[[1, x^2, x^5], Counter({-1: 32, 1: 24, -7: 4, 9: 2, 41: 1, -23: 1})]
[[1, x, x^2, x^5], Counter({-1: 32, 1: 24, -7: 7, 57: 1})]
[[1, x^3, x^5], Counter({-1: 32, 1: 16, -3: 8, 5: 6, 37: 1, -27: 1})]
[[1, x, x^3, x^5], Counter({-1: 32, 1: 16, -3: 8, 5: 4, -11: 3, 53: 1})]
[[1, x^2, x^3, x^5], Counter({-1: 32, 1: 16, -3: 12, 13: 2, 45: 1, -19: 1})]
[[1, x, x^2, x^3, x^5], Counter({-1: 32, 1: 16, -3: 15, 61: 1})]
8


In [33]:
F.<a> = GF(2)
R.<x> = PolynomialRing(F)
n = 7
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
count = 0     
for item in result:
    if 1 in item[1] and 1 in item[0]:
        print(item)
        count +=1
print(count)

[[1, x^6], Counter({-1: 64, 1: 62, 65: 1, -63: 1})]
[[1, x, x^6], Counter({-1: 64, 1: 60, -31: 3, 97: 1})]
[[1, x^2, x^6], Counter({-1: 64, 1: 56, -15: 4, 17: 2, 81: 1, -47: 1})]
[[1, x, x^2, x^6], Counter({-1: 64, 1: 56, -15: 7, 113: 1})]
[[1, x^3, x^6], Counter({-1: 64, 1: 48, -7: 8, 9: 6, 73: 1, -55: 1})]
[[1, x, x^3, x^6], Counter({-1: 64, 1: 48, -7: 8, 9: 4, -23: 3, 105: 1})]
[[1, x^2, x^3, x^6], Counter({-1: 64, 1: 48, -7: 12, 25: 2, 89: 1, -39: 1})]
[[1, x, x^2, x^3, x^6], Counter({-1: 64, 1: 48, -7: 15, 121: 1})]
[[1, x^4, x^6], Counter({-1: 64, 1: 32, -3: 16, 5: 14, 69: 1, -59: 1})]
[[1, x, x^4, x^6], Counter({-1: 64, 1: 32, -3: 16, 5: 12, -27: 3, 101: 1})]
[[1, x^2, x^4, x^6], Counter({-1: 64, 1: 32, -3: 16, 5: 8, -11: 4, 21: 2, 85: 1, -43: 1})]
[[1, x, x^2, x^4, x^6], Counter({-1: 64, 1: 32, -3: 16, 5: 8, -11: 7, 117: 1})]
[[1, x^3, x^4, x^6], Counter({-1: 64, 1: 32, -3: 24, 13: 6, 77: 1, -51: 1})]
[[1, x, x^3, x^4, x^6], Counter({-1: 64, 1: 32, -3: 24, 13: 4, -19: 3, 109: 1

In [36]:
F.<a> = GF(2)
R.<x> = PolynomialRing(F)
n = 4
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)[1:]
result = []
for D in all_D:
    result.append([D, spectra_gcd_graph(f,R,D)])
count = 0     
for item in result:
    if 1 in item[1] and 1 in item[0]:
        print(item)
        count +=1
print(count)

[[1, x^3], Counter({-1: 8, 1: 6, 9: 1, -7: 1})]
[[1, x, x^3], Counter({-1: 8, 1: 4, -3: 3, 13: 1})]
2


In [8]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 5 
f = x^n
D = [1,x,x^2, x^4]
g = abstract_gcd_cayley(f,R,D)
cycles = nx.cycle_basis(g)
result =[]
for cycle in cycles:
    if len(cycle) % 2 == 1 and len(cycle)>3:
        result.append(cycle)
        print(cycle)
        break
print(result)
if len(result) > 0:
    cycle = result[0]
    n = f.degree()-1
    vertice = elements_quotient_ring = list(R.polynomials(max_degree=n))
    v = [vertice[item] for item in cycle]
    print(v)
    cycle = result[0]
    c = induced_graph(g,cycle)
    nx.draw(c)  
else: 
    print("No odd-cycle")
  

[]
No odd-cycle


In [45]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 5 
f = x^n
D = [1,x^4]
g = abstract_gcd_cayley(f,R,D)
cycles = nx.cycle_basis(g)

In [12]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 5 
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)
result = []
for D in all_D:
    g = abstract_gcd_cayley(f,R,D)
    if existence_odd_cycle(g):
        print(D)

In [14]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 3
f = (x^2+1)^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)
result = []
for D in all_D:
    g = abstract_gcd_cayley(f,R,D)
    if existence_odd_cycle(g):
        print(D)

# Which integer could be an eigenvalue for $G_{f}(D)$

In [18]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
n = 3
f = x^n
list_divisors = all_divisors(f)
all_D = powerset(list_divisors)
eig_set = set()
result = []
for D in all_D:
    g = abstract_gcd_cayley(f,R,D)
    spec = spectra_gcd_graph(f,R,D).keys()
    for item in spec:
        eig_set.add(item)
eig_set

{-9, -7, -3, -1, 0, 2, 6, 8, 18, 20, 24, 26}

In [24]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
my_range = range(2,6)
for n in my_range:
    f = x^n
    list_divisors = all_divisors(f)
    all_D = powerset(list_divisors)
    eig_set = set()
    result = []
    for D in all_D:
        g = abstract_gcd_cayley(f,R,D)
        spec = spectra_gcd_graph(f,R,D).keys()
        for item in spec:
            eig_set.add(item)
print(sorted(eig_set))


[-81, -79, -75, -73, -63, -61, -57, -55, -27, -25, -21, -19, -9, -7, -3, -1, 0, 2, 6, 8, 18, 20, 24, 26, 54, 56, 60, 62, 72, 74, 78, 80, 162, 164, 168, 170, 180, 182, 186, 188, 216, 218, 222, 224, 234, 236, 240, 242]


In [6]:
F.<a> = GF(3)
R.<x> = PolynomialRing(F)
eig_set = set()
my_range = range(1,4)
for n in my_range:
    f = (x^2+1)^n
    list_divisors = all_divisors(f)
    all_D = powerset(list_divisors)
    eig_set = set()
    result = []
    for D in all_D:
        g = abstract_gcd_cayley(f,R,D)
        spec = spectra_gcd_graph(f,R,D).keys()
        for item in spec:
            eig_set.add(item)
print(sorted(eig_set))

[-81, -73, -9, -1, 0, 8, 72, 80, 648, 656, 720, 728]


In [7]:
F.<a> = GF(5)
R.<x> = PolynomialRing(F)
eig_set = set()
my_range = range(1,4)
for n in my_range:
    f = x^n
    list_divisors = all_divisors(f)
    all_D = powerset(list_divisors)
    eig_set = set()
    result = []
    for D in all_D:
        g = abstract_gcd_cayley(f,R,D)
        spec = spectra_gcd_graph(f,R,D).keys()
        for item in spec:
            eig_set.add(item)
print(sorted(eig_set))

[-25, -21, -5, -1, 0, 4, 20, 24, 100, 104, 120, 124]


In [12]:
F.<a> = GF(7)
R.<x> = PolynomialRing(F)
eig_set = set()
my_range = range(1,4)
for n in my_range:
    f = x^n
    list_divisors = all_divisors(f)
    all_D = powerset(list_divisors)
    eig_set = set()
    result = []
    for D in all_D:
        g = abstract_gcd_cayley(f,R,D)
        spec = spectra_gcd_graph(f,R,D).keys()
        for item in spec:
            eig_set.add(item)
print(sorted(eig_set))

[-49, -43, -7, -1, 0, 6, 42, 48, 294, 300, 336, 342]


In [10]:
F.<a> = GF(2)
R.<x> = PolynomialRing(F)
eig_set = set()
my_range = range(1,5)
for n in my_range:
    f = x^n
    list_divisors = all_divisors(f)
    all_D = powerset(list_divisors)
    eig_set = set()
    result = []
    for D in all_D:
        g = abstract_gcd_cayley(f,R,D)
        spec = spectra_gcd_graph(f,R,D).keys()
        for item in spec:
            eig_set.add(item)
print(sorted(eig_set))

[-8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [25]:
q = 5
F.<a> = GF(q)
R.<x> = PolynomialRing(F)
eig_set = set()
my_range = range(1,5)
for n in my_range:
    f = x^n
    list_divisors = all_divisors(f)
    all_D = powerset(list_divisors)
    eig_set = set()
    result = []
    for D in all_D:
        g = abstract_gcd_cayley(f,R,D)
        spec = spectra_gcd_graph(f,R,D).keys()
        for item in spec:
            if item > 0:
                eig_set.add(item//(q-1))
            else:
                eig_set.add((item+1)//(q-1))
print(sorted(eig_set))
for item in eig_set:
    print(abs(item).digits(5))

[-31, -30, -26, -25, -6, -5, -1, 0, 1, 5, 6, 25, 26, 30, 31, 125, 126, 130, 131, 150, 151, 155, 156]
[]
[1]
[0, 1, 0, 1]
[1, 1, 0, 1]
[0, 1]
[1, 1]
[0, 0, 1, 1]
[1, 0, 1, 1]
[0, 0, 1]
[1, 0, 1]
[0, 1, 1, 1]
[1, 1, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[1, 0, 1]
[0, 0, 1]
[1]
[1, 1]
[0, 1]
[0, 0, 0, 1]
[1, 0, 0, 1]


In [26]:
q = 3
F.<a> = GF(q)
R.<x> = PolynomialRing(F)
eig_set = set()
my_range = range(1,5)
for n in my_range:
    f = x^n
    list_divisors = all_divisors(f)
    all_D = powerset(list_divisors)
    eig_set = set()
    result = []
    for D in all_D:
        g = abstract_gcd_cayley(f,R,D)
        spec = spectra_gcd_graph(f,R,D).keys()
        for item in spec:
            if item > 0:
                eig_set.add(item//(q-1))
            else:
                eig_set.add((item+1)//(q-1))
print(sorted(eig_set))
for item in eig_set:
    print(abs(item).digits(q))

[-13, -12, -10, -9, -4, -3, -1, 0, 1, 3, 4, 9, 10, 12, 13, 27, 28, 30, 31, 36, 37, 39, 40]
[]
[1]
[0, 1]
[1, 1]
[0, 0, 1]
[1, 0, 1]
[0, 1, 1]
[1, 1, 1]
[0, 0, 0, 1]
[1, 0, 0, 1]
[0, 1, 0, 1]
[1, 1, 0, 1]
[0, 0, 1, 1]
[1, 0, 1, 1]
[0, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[1, 0, 1]
[0, 0, 1]
[1, 1]
[0, 1]
[1]


In [27]:
q = 4
F.<a> = GF(q)
R.<x> = PolynomialRing(F)
eig_set = set()
my_range = range(1,5)
for n in my_range:
    f = x^n
    list_divisors = all_divisors(f)
    all_D = powerset(list_divisors)
    eig_set = set()
    result = []
    for D in all_D:
        g = abstract_gcd_cayley(f,R,D)
        spec = spectra_gcd_graph(f,R,D).keys()
        for item in spec:
            if item > 0:
                eig_set.add(item//(q-1))
            else:
                eig_set.add((item+1)//(q-1))
print(sorted(eig_set))
for item in eig_set:
    print(abs(item).digits(q))

[-21, -20, -17, -16, -5, -4, -1, 0, 1, 4, 5, 16, 17, 20, 21, 64, 65, 68, 69, 80, 81, 84, 85]
[]
[1]
[0, 1]
[1, 1]
[0, 0, 1]
[1, 0, 1]
[0, 1, 1]
[1, 1, 1]
[0, 0, 0, 1]
[1, 0, 0, 1]
[0, 1, 0, 1]
[1, 1, 0, 1]
[0, 0, 1, 1]
[1, 0, 1, 1]
[0, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[1, 0, 1]
[0, 0, 1]
[1, 1]
[0, 1]
[1]


In [28]:
q = 7
F.<a> = GF(q)
R.<x> = PolynomialRing(F)
eig_set = set()
my_range = range(1,5)
for n in my_range:
    f = x^n
    list_divisors = all_divisors(f)
    all_D = powerset(list_divisors)
    eig_set = set()
    result = []
    for D in all_D:
        g = abstract_gcd_cayley(f,R,D)
        spec = spectra_gcd_graph(f,R,D).keys()
        for item in spec:
            if item > 0:
                eig_set.add(item//(q-1))
            else:
                eig_set.add((item+1)//(q-1))
print(sorted(eig_set))
for item in eig_set:
    print(abs(item).digits(q))

[-57, -56, -50, -49, -8, -7, -1, 0, 1, 7, 8, 49, 50, 56, 57, 343, 344, 350, 351, 392, 393, 399, 400]
[]
[1]
[0, 1]
[0, 0, 1, 1]
[1, 0, 1, 1]
[1, 1]
[0, 1, 1, 1]
[1, 1, 1, 1]
[0, 0, 1]
[1, 0, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[1, 0, 1]
[0, 0, 1]
[0, 0, 0, 1]
[1, 0, 0, 1]
[0, 1, 0, 1]
[1, 1, 0, 1]
[1, 1]
[0, 1]
[1]
